# CDS services in Jupyter notebooks
<span style='color:blue'> Welcome to a Jupyter notebook demonstrating how to access some CDS services with Python. Here you will learn how to: </span>

- find datasets on the **CDS MOC Server**
- use MOCs 
- visulalize and interact with them in the **Aladin Lite** widget 

Find more tutorials @ https://github.com/cds-astro

## MOCs : Multi-Order Coverage maps

- IVOA	standard to	describe arbitrary sky regions
- based on HEALPix sky tessellation	

    - footprints of surveys
    - arbitrary polygons from a list of sky coordinates
    - sky coverage of catalogues

- Space and Time coverages


Check the MOCPy's documentation @ https://cds-astro.github.io/mocpy/ 

In [93]:
from astroquery.cds import cds
import mocpy 

In [92]:
info_1 = cds.find_datasets(meta_data="ID=*SDSS*")
print(info_1['ID']) 


info_2 = cds.find_datasets(meta_data="ID=*XMM*")
print(info_2['ID']) 

            ID           
-------------------------
CDS/J/ApJ/749/10/SDSS-obs
          CDS/P/HLA/SDSSg
          CDS/P/HLA/SDSSr
          CDS/P/HLA/SDSSz
          CDS/P/HST/SDSSg
          CDS/P/HST/SDSSr
          CDS/P/HST/SDSSz
        CDS/P/SDSS9/color
    CDS/P/SDSS9/color-alt
            CDS/P/SDSS9/g
            CDS/P/SDSS9/i
            CDS/P/SDSS9/r
            CDS/P/SDSS9/u
            CDS/P/SDSS9/z
          ID          
----------------------
  ESAVO/P/XMM/EPIC-RGB
ESAVO/P/XMM/OM-Optical
     ESAVO/P/XMM/OM-UV
 xcatdb/P/XMM/PN/color
   xcatdb/P/XMM/PN/eb2
   xcatdb/P/XMM/PN/eb3
   xcatdb/P/XMM/PN/eb4


In [38]:
moc_sdss = cds.find_datasets(meta_data="ID=CDS/P/SDSS9/color*", return_moc=True)
moc_xmm = cds.find_datasets(meta_data="ID=ESAVO/P/XMM/EPIC-RGB", return_moc=True)

In [37]:
from astroquery.simbad import Simbad

m87 = Simbad.query_object('M87')
m87

MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE
,"""h:m:s""","""d:m:s""",,,mas,mas,deg,,,
object,str13,str13,int16,int16,float32,float32,int16,str1,str1,object
M 87,12 30 49.4233,+12 23 28.043,12,12,0.041,0.043,90,A,R,2015AJ....150...58F


Q: Is a given source observed by a specific survey? 

In [50]:
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle

ra, dec = Angle('12h30m49.4233s'), Angle('12d23m28.043s')

moc_xmm.contains(ra, dec)

array([ True])

Q: What fraction of the sky is covered by both SDSS and XMM ?

In [39]:
moc_intersection = moc_sdss.intersection(moc_xmm)

sky_frac = moc_intersection.sky_fraction
print('The intersection of SDSS and XMM covers {:.1f} % of the sky'.format(sky_frac * 100.))

The intersection of SDSS and XMM covers 3.9 % of the sky


In [57]:
import ipyaladin.aladin_widget as ipyal

aladin = ipyal.Aladin(target='M87', survey='P/SDSS9/color')
aladin

Aladin(options=['allow_full_zoomout', 'coo_frame', 'fov', 'full_screen', 'log', 'overlay_survey', 'overlay_sur…

In [58]:
aladin.add_moc_from_dict(moc_sdss.serialize(format='json'), {'color': 'yellow', 'opacity': 0.3, 'name': 'SDSS'})
aladin.add_moc_from_dict(moc_xmm.serialize(format='json'), {'color': 'blue', 'opacity': 0.5, 'name': 'XMM'})
aladin.add_moc_from_dict(moc_intersection.serialize(format='json'), {'color': 'magenta', 'opacity': 0.5, 'name': 'intersection'})

Q: I have a catalogue of sources, how I can select those observed in SDSS and with XMM?

In [48]:
import pyvo 

tap_vizier = pyvo.dal.TAPService('http://tapvizier.u-strasbg.fr/TAPVizieR/tap')

query = """SELECT  *  FROM "VII/192/arplist" """

table_arplist = tap_vizier.search(query).to_table()
table_arplist

recno,Arp,Name,VT,u_VT,dim1,dim2,u_dim2,MType,Uchart,RAJ2000,DEJ2000
,,,mag,,arcmin,arcmin,,,,deg,deg
int32,int16,object,float64,str1,float64,float64,str1,object,int16,float64,float64
112,333,NGC 1024,12.1,,3.9,1.4,,(R')SA(r)ab,175,39.80083333333333,10.847222222222221
120,200,NGC 1134,12.1,,2.5,0.9,,S?,175,43.42124999999999,13.015277777777776
119,190,UGC 02320,15.2,,0.5,0.3,,Multiple Sys,--,42.58291666666666,12.889444444444443
89,290,IC 0195,14.3,,1.6,0.8,,S0,--,30.935833333333328,14.708611111111109
90,290,IC 0196,14.2,,2.8,1.4,,S0-,--,30.95833333333333,14.73972222222222
111,258,UGC 02140A,15.5,,0.9,0.2,,SB.0*/,--,39.78874999999999,18.367499999999996
110,258,UGC 02140,15.4,,1.7,0.7,,IBS9P,--,39.77583333333333,18.382777777777775
109,258,Hickson 18C,16.1,,--,--,,S?,--,39.774583333333325,18.388333333333332


In [51]:
coords = SkyCoord(ra=table_arplist['RAJ2000'], 
                  dec=table_arplist['DEJ2000'], 
                  unit=(u.deg, u.deg))

indexes = moc_intersection.contains(coords.ra, 
                                    coords.dec)

In [53]:
print('{} peculiar galaxies are observed both by SDSS and XMM'.format(len(table_arplist[indexes])))

164 peculiar galaxies are observed both by SDSS and XMM


In [62]:
aladin.add_table(table_arplist[indexes])